In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 50
learning_rate = 0.003

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
# Bidirectional recurrent neural network (many-to-one)
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [7]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/50], Step [100/600], Loss: 0.0669
Epoch [1/50], Step [200/600], Loss: 0.1369
Epoch [1/50], Step [300/600], Loss: 0.1123
Epoch [1/50], Step [400/600], Loss: 0.0328
Epoch [1/50], Step [500/600], Loss: 0.0863
Epoch [1/50], Step [600/600], Loss: 0.0191
Epoch [2/50], Step [100/600], Loss: 0.0391
Epoch [2/50], Step [200/600], Loss: 0.0276
Epoch [2/50], Step [300/600], Loss: 0.1112
Epoch [2/50], Step [400/600], Loss: 0.0299
Epoch [2/50], Step [500/600], Loss: 0.0371
Epoch [2/50], Step [600/600], Loss: 0.0344
Epoch [3/50], Step [100/600], Loss: 0.0225
Epoch [3/50], Step [200/600], Loss: 0.0926
Epoch [3/50], Step [300/600], Loss: 0.0395
Epoch [3/50], Step [400/600], Loss: 0.0414
Epoch [3/50], Step [500/600], Loss: 0.0250
Epoch [3/50], Step [600/600], Loss: 0.0391
Epoch [4/50], Step [100/600], Loss: 0.0261
Epoch [4/50], Step [200/600], Loss: 0.1638
Epoch [4/50], Step [300/600], Loss: 0.0159
Epoch [4/50], Step [400/600], Loss: 0.0343
Epoch [4/50], Step [500/600], Loss: 0.0125
Epoch [4/50

In [12]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'bi_lstm.ckpt')

Test Accuracy of the model on the 10000 test images: 99.03 %
